In [1]:
%matplotlib inline
import matplotlib
import cv2, lmdb, os
import numpy as np
import msgpack
import msgpack_numpy
from matplotlib import pyplot as plt

plt.style.use("ggplot")
msgpack_numpy.patch()

In [2]:
import FileIO as fio
import Execution as exe

In [3]:
from tensorpack import *
from tensorpack.utils.utils import get_tqdm
from tensorpack.utils import logger

In [4]:
class MetaReader:
    def __init__(self, path):
        self.db = lmdb.open(path, subdir=False, map_size=1099511627776 * 2, readonly=True,
                            meminit=False, map_async=True)
        
    def get(self, imgid):
        k = u'{:0>10}'.format(imgid).encode('ascii')
        with self.db.begin() as txn:
            with txn.cursor() as cur:
                return msgpack.loads(cur.get(k))
            
    def __iter__(self):
        with self.db.begin() as txn:
            with txn.cursor() as cur:
                for k, v in cur:
                    yield k, msgpack.loads(v)
    
    def close(self):
        self.db.close()

In [5]:
meta_path = "/media/GPAmaster/GPA_master/new_meta.mdb"
meta_io = MetaReader(meta_path)

In [6]:
rt_path = "/media/GPAmaster/GPA_master"
dst_path_l = ["/media/theta_scratch2/GPA1.0"]

In [7]:
for d_path in dst_path_l:
    exe.run_command(["mkdir", "{:}/img".format(d_path)])

In [7]:
def imgf_gen():
    for k, v in meta_io:
        imgid, subj, takename, cam, src_file, dst_file, cam_ts, mocap_ts, sess_date, cam_param_f, markers, joints, geo_name, is_gpa1, is_test = v
        if is_gpa1:
            from_file = "{:}/{:}".format(rt_path, dst_file.decode())
            with open(from_file, "rb") as in_f:
                in_bytes = in_f.read()
            for d_path in dst_path_l:
                to_file = "{:}/img/{:}".format(d_path, os.path.basename(from_file))
                yield in_bytes, to_file

In [8]:
def copier(dp):
    in_bytes, to_f = dp
    with open(to_f, "wb") as out_f:
        out_f.write(in_bytes)
        return 0

In [9]:
in_df = dataflow.DataFromGenerator(imgf_gen())
mapped_df = dataflow.MultiProcessMapDataZMQ(in_df, 5, copier, strict=True)

In [10]:
with get_tqdm(total=0) as pbar:
    mapped_df.reset_state()
    for _ in iter(mapped_df):
        pbar.update()
    for d_path in dst_path_l:
        exe.run_command(
            ["cp", "-r", "{:}/spacetime_camera".format(rt_path), "{:}/spacetime_camera".format(d_path)])
        exe.run_command(
            ["cp", "{:}/new_meta.mdb".format(rt_path), "{:}/new_meta.mdb".format(d_path)])
    pbar.update()

0it [00:00, ?it/s]

[1107 20:49:42 @argtools.py:146] WRN Install python-prctl so that processes can be cleaned with guarantee.


323797it [2:09:22, 45.64it/s]

[1107 22:59:05 @parallel_map.py:53] ERR [MultiProcessMapDataZMQ] buffer_size cannot be larger than the size of the DataFlow!


324009it [2:09:59, 10.11it/s]
